In [1]:
import tweepy as tp
import pandas as pd
import numpy as np
import re
import nltk
import gensim
import random
import pickle
from datetime import datetime, date, time, timedelta
from keras.models import Model
from keras import optimizers
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors
from keras.models import Model
from keras.layers import Dense, Input, GRU
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
tweets = []
with open ('Fashion_12K_tweets', 'rb') as fp:
    tweets = pickle.load(fp)

In [3]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U0001F1F2-\U0001F1F4"  # Macau flag
                           u"\U0001F1E6-\U0001F1FF"  # flags
                           u"\U0001F600-\U0001F64F"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001F1F2"
                           u"\U0001F1F4"
                           u"\U0001F620"
                           u"\u200d"
                           u"\u2640-\u2642"
                           u"\U0001f98b"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [4]:
http = r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"

tweetSeries = pd.Series(tweets).str.replace(http, "").str.replace(r'\B(\@[ 0-9a-zA-Z_]+\b)(?!;)', "1USER1").str.replace(r'\s\s+', " ")
noHashtag = tweetSeries.str.replace(r'(?<=[\s>])#(\d*[A-Za-z_]+\d*)\b(?!;)', "1HASHTAG1").str.replace(r'\s\s+', " ")
noPunc = noHashtag.str.replace(r'(\.\.\.)', " ").str.replace(r'[.?!:;’<>_\(\)\[\]\{\}\+\=\@\#\$\%\^\|\*\'\"]', "").str.replace(r'\&amp', "").str.replace(r'[–,…\/\-_—0-9]', " ").str.replace(r'[^\x00-\x7F]+', "")
sentList = noPunc.str.replace(" USER", "").str.replace(" HASHTAG", "").str.lower().str.split()
spacePunc = noHashtag.str.replace(http, "").str.replace(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ")

tweetDF = tweetSeries.to_frame("Baseline")
tweetDF["Space B/W Punc"] = spacePunc
tweetDF["List of Words + Punc"] = spacePunc.str.split()
tweetDF["Baseline + No HashTags"] = noHashtag
tweetDF["Baseline + No HashTags + No Punc"] = noPunc
tweetDF["List of Words"] = sentList
tweetDF["Part-of-Speech"] = tweetDF["List of Words"].apply(nltk.pos_tag)
tweetDF

,Baseline,Space B/W Punc,List of Words + Punc,Baseline + No HashTags,Baseline + No HashTags + No Punc,List of Words,Part-of-Speech
0,"The always-on Icon denim jacket, now in new wa...","The always-on Icon denim jacket , now in new w...","[The, always-on, Icon, denim, jacket, ,, now, ...","The always-on Icon denim jacket, now in new wa...",The always on Icon denim jacket now in new wa...,"[the, always, on, icon, denim, jacket, now, in...","[(the, DT), (always, RB), (on, IN), (icon, JJ)..."
1,Our softest sweats ever. An easy vintage vibe ...,Our softest sweats ever . An easy vintage vibe...,"[Our, softest, sweats, ever, ., An, easy, vint...",Our softest sweats ever. An easy vintage vibe ...,Our softest sweats ever An easy vintage vibe f...,"[our, softest, sweats, ever, an, easy, vintage...","[(our, PRP$), (softest, JJS), (sweats, NN), (e..."
2,Layer on a sweatshirt and a wear-everywhere Ic...,Layer on a sweatshirt and a wear-everywhere Ic...,"[Layer, on, a, sweatshirt, and, a, wear-everyw...",Layer on a sweatshirt and a wear-everywhere Ic...,Layer on a sweatshirt and a wear everywhere Ic...,"[layer, on, a, sweatshirt, and, a, wear, every...","[(layer, NN), (on, IN), (a, DT), (sweatshirt, ..."
3,"Yes, you can wear white denim year-round! Our ...","Yes , you can wear white denim year-round ! Ou...","[Yes, ,, you, can, wear, white, denim, year-ro...","Yes, you can wear white denim year-round! Our ...",Yes you can wear white denim year round Our f...,"[yes, you, can, wear, white, denim, year, roun...","[(yes, RB), (you, PRP), (can, MD), (wear, VB),..."
4,How to take your denim game to the next level:...,How to take your denim game to the next level ...,"[How, to, take, your, denim, game, to, the, ne...",How to take your denim game to the next level:...,How to take your denim game to the next level ...,"[how, to, take, your, denim, game, to, the, ne...","[(how, WRB), (to, TO), (take, VB), (your, PRP$..."
5,Universally flattering high waist. Made to hit...,Universally flattering high waist . Made to hi...,"[Universally, flattering, high, waist, ., Made...",Universally flattering high waist. Made to hit...,Universally flattering high waist Made to hit ...,"[universally, flattering, high, waist, made, t...","[(universally, RB), (flattering, VBG), (high, ..."
6,Borrowed from the boys. Here’s how to wear our...,Borrowed from the boys . Here ’ s how to wear ...,"[Borrowed, from, the, boys, ., Here, ’, s, how...",Borrowed from the boys. Here’s how to wear our...,Borrowed from the boys Heres how to wear our b...,"[borrowed, from, the, boys, heres, how, to, we...","[(borrowed, VBN), (from, IN), (the, DT), (boys..."
7,Work to weekend to whatever you've got going o...,Work to weekend to whatever you've got going o...,"[Work, to, weekend, to, whatever, you've, got,...",Work to weekend to whatever you've got going o...,Work to weekend to whatever youve got going on...,"[work, to, weekend, to, whatever, youve, got, ...","[(work, NN), (to, TO), (weekend, NN), (to, TO)..."
8,"This is our bestselling Slim, in new authentic...","This is our bestselling Slim , in new authenti...","[This, is, our, bestselling, Slim, ,, in, new,...","This is our bestselling Slim, in new authentic...",This is our bestselling Slim in new authentic...,"[this, is, our, bestselling, slim, in, new, au...","[(this, DT), (is, VBZ), (our, PRP$), (bestsell..."
9,What's your favorite way to wear #GapDenim: li...,What's your favorite way to wear 1HASHTAG1 : l...,"[What's, your, favorite, way, to, wear, 1HASHT...",What's your favorite way to wear 1HASHTAG1: li...,Whats your favorite way to wear HASHTAG ligh...,"[whats, your, favorite, way, to, wear, light, ...","[(whats, NNS), (your, PRP$), (favorite, JJ), (..."


In [5]:
#Download this from https://nlp.stanford.edu/projects/glove/
#Specifically, Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 25d, 50d, 100d, & 200d vectors, 1.42 GB download): glove.twitter.27B.zip
from gensim.scripts.glove2word2vec import glove2word2vec
filename = "./glove.twitter.27B/glove.twitter.27B.50d.txt"
glove2word2vec(glove_input_file=filename, word2vec_output_file="gensim_glove_vectors.txt")

from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)
words = list(glove_model.wv.vocab)

/Users/anilvadali/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [6]:
wordlist = tweetDF["Part-of-Speech"][13010]
nas = [word.lower() for word,pos in wordlist if (pos.startswith("NN") or pos.startswith("JJ")) and (len(word) > 1)]
nas

['perfect']

In [105]:
pd.set_option('display.max_columns', None)
nounsNadj = []
counter = 0
for wordlist in tweetDF["Part-of-Speech"]:
    nas = [word.lower() for word,pos in wordlist if (pos.startswith("NN") or pos.startswith("JJ")) and (len(word) > 1)] 
    
    if len(nas) >= 3:
        keywords = random.sample(nas, 3)
    elif len(nas) >= 0 and len(tweetDF["List of Words"][counter]) > 2:
        sample = random.sample(tweetDF["List of Words"][counter],3)
        keywords = [word.strip("\“").lower() for word in sample if len(word) > 1]
    elif len(nas) >= 0 and len(tweetDF["List of Words"][counter]) > 1:
        sample = random.sample(tweetDF["List of Words"][counter],2)
        keywords = [word.strip("\“").lower() for word in sample if len(word) > 1]
    elif len(nas) >= 0 and len(tweetDF["List of Words"][counter]) > 0:
        sample = random.sample(tweetDF["List of Words"][counter],1)
        keywords = [word.strip("\“").lower() for word in sample]
    nounsNadj.append(list(set(keywords)))
    counter += 1
    

tweetDF["Keywords"] = nounsNadj
num = 7186
print(re.sub(http, "", tweetDF["Baseline"][num]))
print(tweetDF["Space B/W Punc"][num])
print(tweetDF["List of Words + Punc"][num])
print(tweetDF["Baseline + No HashTags"][num])
print(tweetDF["Baseline + No HashTags + No Punc"][num])
print(tweetDF["List of Words"][num])
print(tweetDF["List of Words + Punc"][num])
print(tweetDF["Part-of-Speech"][num])
print(tweetDF["Keywords"][7100:7200])

tweetDF

one-off vintage graphic tees are here and going fast–snag your fave before it's gone for good.
one-off vintage graphic tees are here and going fast – snag your fave before it's gone for good . 
['one-off', 'vintage', 'graphic', 'tees', 'are', 'here', 'and', 'going', 'fast', '–', 'snag', 'your', 'fave', 'before', "it's", 'gone', 'for', 'good', '.']
one-off vintage graphic tees are here and going fast–snag your fave before it's gone for good.
one off vintage graphic tees are here and going fast snag your fave before its gone for good
['one', 'off', 'vintage', 'graphic', 'tees', 'are', 'here', 'and', 'going', 'fast', 'snag', 'your', 'fave', 'before', 'its', 'gone', 'for', 'good']
['one-off', 'vintage', 'graphic', 'tees', 'are', 'here', 'and', 'going', 'fast', '–', 'snag', 'your', 'fave', 'before', "it's", 'gone', 'for', 'good', '.']
[('one', 'CD'), ('off', 'RP'), ('vintage', 'NN'), ('graphic', 'JJ'), ('tees', 'NNS'), ('are', 'VBP'), ('here', 'RB'), ('and', 'CC'), ('going', 'VBG'), ('fast'

,Baseline,Space B/W Punc,List of Words + Punc,Baseline + No HashTags,Baseline + No HashTags + No Punc,List of Words,Part-of-Speech,Keywords
0,"The always-on Icon denim jacket, now in new wa...","The always-on Icon denim jacket , now in new w...","[The, always-on, Icon, denim, jacket, ,, now, ...","The always-on Icon denim jacket, now in new wa...",The always on Icon denim jacket now in new wa...,"[the, always, on, icon, denim, jacket, now, in...","[(the, DT), (always, RB), (on, IN), (icon, JJ)...","[styles, favorite, icon]"
1,Our softest sweats ever. An easy vintage vibe ...,Our softest sweats ever . An easy vintage vibe...,"[Our, softest, sweats, ever, ., An, easy, vint...",Our softest sweats ever. An easy vintage vibe ...,Our softest sweats ever An easy vintage vibe f...,"[our, softest, sweats, ever, an, easy, vintage...","[(our, PRP$), (softest, JJS), (sweats, NN), (e...","[softest, sweatshirt, colorful]"
2,Layer on a sweatshirt and a wear-everywhere Ic...,Layer on a sweatshirt and a wear-everywhere Ic...,"[Layer, on, a, sweatshirt, and, a, wear-everyw...",Layer on a sweatshirt and a wear-everywhere Ic...,Layer on a sweatshirt and a wear everywhere Ic...,"[layer, on, a, sweatshirt, and, a, wear, every...","[(layer, NN), (on, IN), (a, DT), (sweatshirt, ...","[shop, denim, icon]"
3,"Yes, you can wear white denim year-round! Our ...","Yes , you can wear white denim year-round ! Ou...","[Yes, ,, you, can, wear, white, denim, year-ro...","Yes, you can wear white denim year-round! Our ...",Yes you can wear white denim year round Our f...,"[yes, you, can, wear, white, denim, year, roun...","[(yes, RB), (you, PRP), (can, MD), (wear, VB),...","[chambray, denim, ways]"
4,How to take your denim game to the next level:...,How to take your denim game to the next level ...,"[How, to, take, your, denim, game, to, the, ne...",How to take your denim game to the next level:...,How to take your denim game to the next level ...,"[how, to, take, your, denim, game, to, the, ne...","[(how, WRB), (to, TO), (take, VB), (your, PRP$...","[shop, denim, game]"
5,Universally flattering high waist. Made to hit...,Universally flattering high waist . Made to hi...,"[Universally, flattering, high, waist, ., Made...",Universally flattering high waist. Made to hit...,Universally flattering high waist Made to hit ...,"[universally, flattering, high, waist, made, t...","[(universally, RB), (flattering, VBG), (high, ...","[skinny, rise, notes]"
6,Borrowed from the boys. Here’s how to wear our...,Borrowed from the boys . Here ’ s how to wear ...,"[Borrowed, from, the, boys, ., Here, ’, s, how...",Borrowed from the boys. Here’s how to wear our...,Borrowed from the boys Heres how to wear our b...,"[borrowed, from, the, boys, heres, how, to, we...","[(borrowed, VBN), (from, IN), (the, DT), (boys...","[bold, double, great]"
7,Work to weekend to whatever you've got going o...,Work to weekend to whatever you've got going o...,"[Work, to, weekend, to, whatever, you've, got,...",Work to weekend to whatever you've got going o...,Work to weekend to whatever youve got going on...,"[work, to, weekend, to, whatever, youve, got, ...","[(work, NN), (to, TO), (weekend, NN), (to, TO)...","[youve, work, jeans]"
8,"This is our bestselling Slim, in new authentic...","This is our bestselling Slim , in new authenti...","[This, is, our, bestselling, Slim, ,, in, new,...","This is our bestselling Slim, in new authentic...",This is our bestselling Slim in new authentic...,"[this, is, our, bestselling, slim, in, new, au...","[(this, DT), (is, VBZ), (our, PRP$), (bestsell...","[shop, new, own]"
9,What's your favorite way to wear #GapDenim: li...,What's your favorite way to wear 1HASHTAG1 : l...,"[What's, your, favorite, way, to, wear, 1HASHT...",What's your favorite way to wear 1HASHTAG1: li...,Whats your favorite way to wear HASHTAG ligh...,"[whats, your, favorite, way, to, wear, light, ...","[(whats, NNS), (your, PRP$), (favorite, JJ), (...","[classic, washes, whats]"


In [8]:
" 🥖".encode('unicode-escape').decode('ASCII')

' \\U0001f956'

In [9]:
#Encoding Matrix
word_vec_matrix = np.empty((0,50))
for tweet in tweetDF["Keywords"]:
    vector = np.zeros(50)
    for keyword in tweet:
        try:
            vector += glove_model[keyword.strip("\”")]
        except:
            continue
    word_vec_matrix = np.vstack((word_vec_matrix,vector))

    
word_vec_matrix.shape

(13175, 50)

In [10]:
max_tweet_length = max(tweetDF['List of Words + Punc'].str.len())+2

In [11]:
#Decoding Input Dictionary

dinput_vec_matrix = np.zeros((word_vec_matrix.shape[0],max_tweet_length)).astype(int)
int_to_word = {0: "", 1: "[START]", 2: "[END]"}
word_to_int = {"": 0, "[START]": 1, "[END]": 2}
dict_counter = 3
row_counter = 0

for tweet in tweetDF["List of Words + Punc"]:
    vector = np.zeros(max_tweet_length)
    word_counter = 0
    tweet = ["[START]"] + tweet
    for word in tweet:
        if word not in word_to_int.keys():
            int_to_word[dict_counter] = word
            word_to_int[word] = dict_counter
            
            vector[word_counter] = dict_counter
            dict_counter += 1
            word_counter += 1
        else:
            vector[word_counter] = word_to_int[word]
            word_counter += 1
    
    dinput_vec_matrix[row_counter] = vector
    row_counter += 1

dinput_vec_matrix = dinput_vec_matrix.astype(int)
dinput_vec_matrix.shape

(13175, 69)

In [12]:
doutput_vec_matrix = np.zeros((dinput_vec_matrix.shape[0],max_tweet_length,len(word_to_int.keys())), dtype = int)
row_counter = 0

for tweet in tweetDF["List of Words + Punc"]:
    word_counter = 0
    tweet = tweet + ["[END]"]
    for word in tweet:
        doutput_vec_matrix[row_counter, word_counter, word_to_int[word]] = 1
        word_counter += 1

    row_counter += 1

doutput_vec_matrix.shape

(13175, 69, 14558)

In [13]:
result = glove_model.most_similar(positive=["gorgeous", "sale", "home"], topn=10)
print(result)

[('looking', 0.8687669038772583), ('new', 0.8621575236320496), ('here', 0.8555992245674133), ('look', 0.8554851412773132), ('this', 0.851833164691925), ('from', 0.848891019821167), ('out', 0.8419684171676636), ('today', 0.8406338095664978), ('coming', 0.8352305889129639), ('big', 0.832432746887207)]


In [14]:
from keras.models import Model
from keras.layers import Dense, Input, GRU
from keras.layers.embeddings import Embedding

word_dim = 50
num_tokens = len(word_to_int.keys())

# Define the layers
word_vec_input = Input(shape=(word_dim,))
decoder_inputs = Input(shape=(None,))
decoder_embed = Embedding(input_dim=num_tokens, output_dim=word_dim, mask_zero=True)
decoder_gru_1 = GRU(word_dim, return_sequences=True, return_state=False)
decoder_gru_2 = GRU(word_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(num_tokens, activation='softmax')

# Connect the layers
embedded = decoder_embed(decoder_inputs)
gru_1_output = decoder_gru_1(embedded, initial_state=word_vec_input)
gru_2_output, state_h = decoder_gru_2(gru_1_output)
decoder_outputs = decoder_dense(gru_2_output)

# Define the model that will be used for training
training_model = Model([word_vec_input, decoder_inputs], decoder_outputs)

In [15]:
batch_size = 128
epochs = 50

adam = optimizers.Adam(lr=.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0, amsgrad=False)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
training_model.fit([word_vec_matrix, dinput_vec_matrix], doutput_vec_matrix,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 10540 samples, validate on 2635 samples
Epoch 1/50
10540/10540 [==============================] - 590s 56ms/step - loss: 7.4579 - val_loss: 6.5916
Epoch 2/50
10540/10540 [==============================] - 561s 53ms/step - loss: 6.6902 - val_loss: 6.5892
Epoch 3/50
10540/10540 [==============================] - 602s 57ms/step - loss: 6.5956 - val_loss: 6.4964
Epoch 4/50
10540/10540 [==============================] - 608s 58ms/step - loss: 6.4322 - val_loss: 6.3695
Epoch 5/50
10540/10540 [==============================] - 547s 52ms/step - loss: 6.2973 - val_loss: 6.2490
Epoch 6/50
10540/10540 [==============================] - 535s 51ms/step - loss: 6.1718 - val_loss: 6.1661
Epoch 7/50
10540/10540 [==============================] - 534s 51ms/step - loss: 6.0594 - val_loss: 6.0643
Epoch 8/50
10540/10540 [==============================] - 535s 51ms/step - loss: 5.9519 - val_loss: 5.9785
Epoch 9/50
10540/10540 [==============================] - 532s 51ms/step - loss: 5.8447 - val_l

KeyboardInterrupt: 

In [16]:
# decoder_embed2 = Embedding(input_dim=num_tokens, output_dim=word_dim, mask_zero=True)
# embedded2 = decoder_embed2(decoder_inputs)
# gru_1_output2 = decoder_gru_1(embedded2, initial_state=word_vec_input)
# gru_2_output2, state_h2 = decoder_gru_2(gru_1_output2)
# decoder_outputs2 = decoder_dense(gru_2_output2)

# Also create a model for inference (this returns the GRU state)
decoder_model = Model([word_vec_input, decoder_inputs], [decoder_outputs, state_h])

In [17]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    encoding_vector = np.zeros((1,50))
    for word in input_seq:
        try:
            encoding_vector[0] += glove_model[word]
        except:
            continue
    
#     print(encoding_vector.shape)
            
    target_seq = np.zeros((1,59), dtype=int)
    target_seq[0,0] = word_to_int["[START]"]
#     target_seq[0,1] = word_to_int["We"]
#     target_seq[0,2] = word_to_int["are"]
#     target_seq[0,3] = word_to_int[","]
    
#     print(len(target_seq))
    
    stop_condition = False
    decoded_sentence = ''
    counter = 1
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    while not stop_condition:
        output_tokens, h= decoder_model.predict([encoding_vector, target_seq])
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int_to_word[sampled_token_index]
        decoded_sentence += sampled_char + " "
        
        if (sampled_char == "[END]" or
           len(decoded_sentence.split()) > 37):
            stop_condition = True
        
#         next_word = np.zeros((1,59))
#         next_word[0, counter] = sampled_token_index
        target_seq[0, counter] = sampled_token_index
        counter+=1
    
    
    return decoded_sentence

In [144]:
# print(word_vec_matrix.shape)
# print(dinput_vec_matrix.shape)
# print('-----------')
print(decode_sequence(["small", "dress", "colorful"]))
print(decode_sequence(["fave", "graphic", "tees"]))
print(decode_sequence(["beach", "sunny", "towels"]))
print(decode_sequence(["weekend", "surfing", "trip"]))
print(decode_sequence(["weekend", "sales"]))
print(decode_sequence(["warm", "cozy"]))
print(decode_sequence(["nice", "sun", "friday"]))

The perfect dress of the perfect fit . [END] 
The new collection is here . Shop women's new arrivals : [END] 
Feelin' hot ! Get the look : [END] 
It's Friday , we ’ re ready for the weekend . [END] 
TODAY ONLY ! Snag 50 % off all sale & amp ; online . Shop now : [END] 
The perfect sweater is a good thing ? Get it here : [END] 
A look you need to wear ? 1USER1 [END] 


In [ ]:
#lowercase everything
#word embedding size increase
#increase learning rate
#rnn encoder for word embeddings
#smaller dataset to overfitting